#### 刷涂鸦自动脚本
>1 选好要刷的涂鸦|难度|倍率  
2 截图识别文字有"开始调查",点击  
3 识别有文字放弃，点击屏幕中间  
4 识别有“一键上阵”，点击这个位置，等待1秒，再点击确认位置  
5 等待2分钟后，循环识别“退出” 点击这个位置  
6 循环等待“开始调查”，点击这个位置

In [1]:
import time
from io import BytesIO
from PIL import ImageGrab, Image
import pyautogui
import ddddocr

In [2]:
class ImageOrc:
    '''
        从PIL截图的数据转为识别的文字
        image是PIL工具直接截取的数据，如果是已有图片，需要先打开
    '''
    def __init__(self):       
        self.ocr = ddddocr.DdddOcr()  # 识别实列
    
    def trans_orc(self, image):
        img_byte = BytesIO()  # IO实列 每次都要重新开一个实例
        image.save(img_byte, format='JPEG') # 把图片存到内存 format: PNG or JPEG
        binary_content = img_byte.getvalue()  # 把内存里面的im对象转为二进制流
        try:
            res = self.ocr.classification(binary_content)  # 转成文字
            return res
        except:
            print('转换文字失败')

kaishidiaocha = (911, 520, 1046, 560)  # 开始调查
yijianshangzhen = (975, 460, 1060, 488)  # 一键上阵
fangqi = (939, 534, 1055, 566)  # 放弃
tuichu = (917, 522, 983, 557)  # 退出

orc = ImageOrc()

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder


In [3]:
def in_start():
    '''
        从开始界面能进入到地图页面
    '''
    im1 = ImageGrab.grab(bbox=kaishidiaocha)  # 左上角 右下角 
    t1 = orc.trans_orc(im1)

    if t1 == '开始调查':
        pyautogui.moveTo(982, 538)  # 点击‘开始调查’
        pyautogui.click()
        
        time.sleep(1)

        while 1:
            im0 = ImageGrab.grab(bbox=fangqi)  # 截取‘放弃’位置图片
            t0 = orc.trans_orc(im0)
            
            if t0 == '放弃':
                break
                
            pyautogui.moveTo(982, 538)  # 如果没点进去 重新点击‘开始调查’
            pyautogui.click()  
            
def in_end_war():
    '''
        进入最后的战斗
    '''
    pyautogui.moveTo(590, 300)  # 点击屏幕中间
    pyautogui.click()
        
    time.sleep(1)

    while 1:   
        im2 = ImageGrab.grab(bbox=yijianshangzhen)  # 检查一键上阵按钮 
        t2 = orc.trans_orc(im2)

        if t2 == '一键上阵':
            break

        pyautogui.moveTo(590, 300)  # 如果没点进去，重新点击屏幕中间
        pyautogui.click()

        
def start_war():
    '''
        进入战斗界面，开始战斗
    '''
    while 1:
        pyautogui.moveTo(1019, 477)  # 点击‘一键上阵’
        pyautogui.click()

        time.sleep(1)

        pyautogui.moveTo(966, 560)  #  点击‘开始’
        pyautogui.click()
        
        time.sleep(1)

        im2 = ImageGrab.grab(bbox=yijianshangzhen)  # 再次检查一键上阵按钮
        t2 = orc.trans_orc(im2)

        if t2 != '一键上阵':  # 一键上阵按钮 不存在说明游戏开始了
            break

            
def exit_click():
    '''
        识别、点击退出按钮
    '''
    im3 = ImageGrab.grab(bbox=tuichu)  # 左上角 右下角 
    t3 = orc.trans_orc(im3)

    if t3 == '退出':
        time.sleep(5)
        pyautogui.moveTo(951, 536)  # 点击‘退出’
        pyautogui.click()
        

def ensure_station_exit():
    while 1:
        exit_click()  # 点击退出
        
        im0 = ImageGrab.grab(bbox=fangqi)  # 截取‘放弃’位置图片
        t0 = orc.trans_orc(im0)

        if t0 == '放弃':
            break
            
        time.sleep(5)

In [6]:
ox = 2170 // 120 # 剩余票 整除 消费票数

for _ in range(ox):
    in_start()  # 进入到地图界面
    time.sleep(1)
    
    #-----------拿支援点位--8级点--------
    pyautogui.moveTo(822, 323)  # 点位1
    pyautogui.click()
    start_war()  # 开始战斗
    time.sleep(30)
    ensure_station_exit()  # 退出到地图界面
    time.sleep(2)
    
    pyautogui.moveTo(818, 439)  # 点位2
    pyautogui.click() 
    start_war()  # 开始战斗
    time.sleep(30)
    ensure_station_exit()  # 退出到地图界面
    time.sleep(2)
    #-----------拿支援点位--8级点--------
    
    in_end_war()  # 进入到终战
    time.sleep(1)
    start_war()  # 开始战斗
    time.sleep(60)
    
    while 1:  # 退出机制
        pyautogui.moveTo(582, 532)  # 点击空白处 防升级遮住
        pyautogui.click()  
        
        time.sleep(2)
        
        exit_click()  # 退出按钮

        # 确认退出成功
        im1 = ImageGrab.grab(bbox=kaishidiaocha)  # 左上角 右下角 
        t1 = orc.trans_orc(im1)

        if t1 == '开始调查':
            break
        
        # 如果失败，会退回到全局页面
        im0 = ImageGrab.grab(bbox=fangqi)  # 截取‘放弃’位置图片
        t0 = orc.trans_orc(im0)
        
        if t0 == '放弃':  # 重新打
            in_end_war() # 进入到终战
            time.sleep(1)
            start_war()  # 开始战斗
            time.sleep(60)
            
        time.sleep(5)